In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/ramen-ratings/ramen-ratings.csv')
data.info()

In [ ]:
data.isnull().any()

In [ ]:
data['Style'].fillna('Unknown', inplace=True)
data['Country'][573]='USA'

In [ ]:
data.head()

Number of Variety

In [ ]:
number_of_variety_by_style = data.groupby(['Country','Style']).size().unstack().fillna(0).astype('int64').reset_index()
number_of_variety_by_style['Total'] = number_of_variety_by_style.sum(axis=1).astype('int64')
number_of_variety_by_style = number_of_variety_by_style.sort_values(by='Total', ascending=False)

In [ ]:
import plotly.express as px

fig = px.bar(number_of_variety_by_style, 
             x='Country', 
             y=['Bar','Bowl','Box','Can','Cup','Pack','Tray','Unknown'],
             labels={'value':'Number of Variety'},
             range_x=[-0.5,9.5],
             title='Top 10 Countries by Number of Variety')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Number of Brand

In [ ]:
number_of_brand = data.groupby(['Country','Brand']).size().to_frame().reset_index()
number_of_brand = number_of_brand['Country'].value_counts().to_frame().reset_index()
number_of_brand.columns = ['Country','Number of Brand']

In [ ]:
fig2 = px.bar(number_of_brand, 
              x='Country', 
              y='Number of Brand', 
              title='Top 10 Countries by Number of Brand', 
              range_x=[-0.5,9.5],
              range_y=[0,65],
              text='Number of Brand')
fig2.update_traces(textposition='outside')
fig2.show()

Top Ramen

In [ ]:
top_ramen = data[data['Top Ten'].isna() == False]
top_ramen['Top Ten'].unique()

In [ ]:
top_ramen = top_ramen[top_ramen['Top Ten'] != '\n']
top_ramen[['Year Awarded','Rank']] = top_ramen['Top Ten'].str.split('#', expand=True)

In [ ]:
top_ramen['Year Awarded'] = top_ramen['Year Awarded'].astype('int64')
top_ramen['Rank'] = top_ramen['Rank'].astype('int64')

In [ ]:
fig4 = px.scatter(top_ramen, 
                  x="Year Awarded", 
                  y="Rank",
                  range_x = [2011.5,2016.5],
                  range_y = [10.7,0.3],
                  hover_name = 'Variety',
                  hover_data=['Country'],
                  color='Brand', 
                  color_discrete_sequence=px.colors.qualitative.Alphabet,
                  title='Ramen Ranks by Year', 
                  text='Brand')
fig4.update_traces(marker=dict(size=5), textposition='bottom center')
fig4.update_xaxes(showgrid=False)
fig4.update_yaxes(showgrid=True, gridwidth=31)
fig4.update_layout(yaxis = dict(tickmode = 'linear',tick0 = 1,dtick = 1))
fig4.show()

Indonesian High Rated Brands

In [ ]:
brand_indonesia = data[data['Country']=='Indonesia']
brand_indonesia['Brand'].replace('Mi Sedaap','Mie Sedaap', inplace=True)

In [ ]:
brand_indonesia['Stars'] = brand_indonesia['Stars'].astype('float64')
brand_indonesia = brand_indonesia.groupby('Brand').agg({'Stars':'mean', 'Brand':'size'}).rename(columns={'Brand':'Number of Variety'}).reset_index()
brand_indonesia['Stars'] = brand_indonesia['Stars'].round(2)
brand_indonesia.head()

In [ ]:
fig5 = px.bar(brand_indonesia, 
              x='Brand', 
              y='Stars',
              range_y=[2.7,5],
              color='Number of Variety',
              color_continuous_scale ='aggrnyl',
              text='Stars',
              title='Indonesian Ramen Brand Average Rating')
fig5.update_layout(xaxis={'categoryorder':'total descending'})
fig5.update_traces(textposition='outside')
fig5.show()

Indonesian Ramen Brand Style

In [ ]:
indo_brand_style = data[data['Country']=='Indonesia'].groupby(['Brand','Style']).size().unstack().reset_index()
indo_brand_style = indo_brand_style.fillna(0)
indo_brand_style.head()

In [ ]:
fig6 = px.bar(indo_brand_style, 
              x='Brand', 
              y=['Box','Cup','Pack'],
              labels={'value':'Number of Variety'},
              range_y=[0,15],
              title='Indonesian Ramen Brand Style')
fig6.update_layout(xaxis={'categoryorder':'total descending'})
fig6.show()

In [ ]:
indo_style = data[data['Country']=='Indonesia'].groupby('Style').size().reset_index()
indo_style.columns = ['Style','Count']

In [ ]:
fig7 = px.pie(indo_style, 
              values='Count', 
              names='Style',
              color_discrete_sequence=px.colors.qualitative.Set3,
              title='Indonesian Ramen Style')
fig7.update_traces(textposition='outside', textinfo='percent+label')
fig7.show()

**--**